In [1]:
import numpy as np
import pandas as pd
from pandas import DataFrame
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import *
from sklearn.preprocessing import label_binarize
from tqdm import tqdm

In [2]:
def feature_vector(feature_name, df, similarity='jaccard'):
    def Jaccard(matrix):
        matrix = np.mat(matrix)

        numerator = matrix * matrix.T

        denominator = (
            np.ones(np.shape(matrix)) * matrix.T
            + matrix * np.ones(np.shape(matrix.T))
            - matrix * matrix.T
        )

        return numerator / denominator
    
    def Cosine(matrix):
        norms = np.linalg.norm(matrix, axis=1)
        matrix_normalized = matrix / norms[:, np.newaxis]
        return np.dot(matrix_normalized, matrix_normalized.T)

    
    def Gaussian(matrix, gamma=None):
        if gamma is None:
            norms = np.linalg.norm(matrix, axis=1)
            avg_norm = np.mean(norms)
            gamma = 1 / avg_norm**2
        
        squared_norms = np.sum(matrix**2, axis=1)
        squared_distance = squared_norms[:, np.newaxis] + squared_norms - 2 * np.dot(matrix, matrix.T)
        
        return np.exp(-gamma * squared_distance)

    all_feature = []
    drug_list = np.array(df[feature_name]).tolist()
    # Features for each drug, for example, when feature_name is target, drug_list=["P30556|P05412","P28223|P46098|……"]
    for i in drug_list:
        for each_feature in i.split("|"):
            if each_feature not in all_feature:
                all_feature.append(each_feature)  # obtain all the features
    #print("length of all feature is", len(all_feature))
    feature_matrix = np.zeros((len(drug_list), len(all_feature)), dtype=float)
    df_feature = DataFrame(
        feature_matrix, columns=all_feature
    )  # Consrtuct feature matrices with key of dataframe
    for i in range(len(drug_list)):
        for each_feature in df[feature_name].iloc[i].split("|"):
            df_feature[each_feature].iloc[i] = 1

    df_feature = np.array(df_feature)
    if similarity == 'jaccard':
        sim_matrix = np.array(Jaccard(df_feature))
    elif similarity == 'cosine':
        sim_matrix = Cosine(df_feature)
    elif similarity == 'gaussian':
        sim_matrix = Gaussian(df_feature)
    else:
        raise ValueError("Unknown similarity measure")
    
    #print(feature_name + " len is:" + str(len(sim_matrix[0])))
    return sim_matrix


def prepare(df_drug, feature_list, mechanism, action, drugA, drugB, similarity='jaccard'):
    d_label = {}
    d_feature = {}

    # Transfrom the interaction event to number
    d_event = []
    for i in range(len(mechanism)):
        d_event.append(mechanism[i] + " " + action[i])

    count = {}
    for i in d_event:
        if i in count:
            count[i] += 1
        else:
            count[i] = 1
    event_num = len(count)
    list1 = sorted(count.items(), key=lambda x: x[1], reverse=True)
    for i in range(len(list1)):
        d_label[list1[i][0]] = i

    vector = []
    for i in feature_list:
        #vector = np.hstack((vector, feature_vector(i, df_drug, vector_size)))
        vector.append(feature_vector(i, df_drug, similarity))
    vector = np.stack(vector, axis=-1)
    # Transfrom the drug ID to feature vector
    for i in range(len(np.array(df_drug["name"]).tolist())):
        d_feature[np.array(df_drug["name"]).tolist()[i]] = vector[i]

    # Use the dictionary to obtain feature vector and label
    new_feature = []
    new_label = []

    for i in range(len(d_event)):
        temp = np.concatenate([d_feature[drugA[i]][None], d_feature[drugB[i]][None]], axis=0)
        new_feature.append(temp)
        new_label.append(d_label[d_event[i]])

    new_feature = np.array(new_feature)  # 323539*....
    new_label = np.array(new_label)  # 323539

    return new_feature, new_label, event_num


df_drug = pd.read_csv("drug_features.csv")
extraction = pd.read_csv("extraction.csv")
mechanism = extraction["mechanism"]
action = extraction["action"]
drugA = extraction["drugA"]
drugB = extraction["drugB"]
feature_list = ["pathway", "target", "enzyme", "category"]
new_feature, new_label, event_num = prepare(df_drug, feature_list, mechanism, action, drugA, drugB, similarity='gaussian')
new_feature = torch.tensor(new_feature, dtype=torch.float32)

In [3]:

class DrugInteractionDataset(Dataset):
    def __init__(self, features, labels):
        self.features = torch.tensor(features, dtype=torch.float32)
        self.labels = torch.tensor(labels, dtype=torch.long)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, index):
        return self.features[index], self.labels[index]

# Split the data into training and validation sets
features_train, features_val, labels_train, labels_val = train_test_split(
    new_feature, new_label, test_size=0.2, random_state=42
)

# Create dataset objects
train_dataset = DrugInteractionDataset(features_train, labels_train)
val_dataset = DrugInteractionDataset(features_val, labels_val)

# Create data loaders
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

/tmp/ipykernel_235039/3982585584.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.features = torch.tensor(features, dtype=torch.float32)


In [4]:


class CNNDDI(nn.Module):
    def __init__(self, num_of_features=4):
        super(CNNDDI, self).__init__()
        self.conv1 = nn.Conv2d(2, 64, (3, 1), padding=(1, 0))
        self.conv2 = nn.Conv2d(64, 128, (3, 1), padding=(1, 0))
        self.conv3_1 = nn.Conv2d(128, 128, (3, 1), padding=(1, 0))
        self.conv3_2 = nn.Conv2d(128, 128, (3, 1), padding=(1, 0))
        self.conv4 = nn.Conv2d(128, 256, (3, 1), padding=(1, 0))
        self.fc1 = nn.Linear(256 * 572 * num_of_features, 256)  # Adjust feature_size based on your input dimensions
        self.fc2 = nn.Linear(256, 65)  # Assuming 65 DDI types

    def forward(self, x):
        x = F.leaky_relu(self.conv1(x), negative_slope=0.2)
        x = F.leaky_relu(self.conv2(x), negative_slope=0.2)
        identity = x
        x = F.leaky_relu(self.conv3_1(x), negative_slope=0.2)
        x = self.conv3_2(x)
        x += identity
        x = F.leaky_relu(x, negative_slope=0.2)
        x = F.leaky_relu(self.conv4(x), negative_slope=0.2)
        x = torch.flatten(x, 1)
        x = F.leaky_relu(self.fc1(x), negative_slope=0.2)
        x = self.fc2(x)
        return x


In [5]:
def train(model, device, train_loader, optimizer, criterion):
    model.train()
    total_loss = 0
    for features, labels in train_loader:
        features, labels = features.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(features)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(train_loader)

def validate(model, device, val_loader, criterion):
    model.eval()
    total_loss = 0
    correct = 0
    total = 0
    all_predicted = []
    all_labels = []
    all_probs = []

    with torch.no_grad():
        for features, labels in val_loader:
            features, labels = features.to(device), labels.to(device)
            outputs = model(features)
            loss = criterion(outputs, labels)
            total_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
            all_predicted.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            all_probs.extend(torch.nn.functional.softmax(outputs.data, 1).cpu().numpy())

    all_predicted = np.array(all_predicted)
    all_labels = np.array(all_labels)
    all_probs = np.array(all_probs)

    val_loss = total_loss / len(val_loader)
    val_accuracy = correct / len(val_loader.dataset)
    precision = precision_score(all_labels, all_predicted, average='macro',zero_division=0)
    recall = recall_score(all_labels, all_predicted, average='macro',zero_division=0)
    f1 = f1_score(all_labels, all_predicted, average='macro',zero_division=0)
    roc_auc = roc_auc_score(label_binarize(all_labels, classes=range(65))[:,1], all_probs[:,1], multi_class='ovo', average='micro')
    aupr = average_precision_score(label_binarize(all_labels, classes=range(65))[:,1], all_probs[:,1], average='micro')

    return val_loss, val_accuracy, precision, recall, f1, roc_auc, aupr

def save_checkpoint(model, optimizer, epoch, filename):
    state = {
        'epoch': epoch,
        'state_dict': model.state_dict(),
        'optimizer': optimizer.state_dict()
    }
    torch.save(state, filename)
    print(f"Saved checkpoint: {filename}")



In [6]:
class focal_loss(nn.Module):
    def __init__(self, gamma=2):
        super(focal_loss, self).__init__()
        self.gamma = gamma

    def forward(self, preds, labels):
        labels = labels[..., None]

        preds_logsoft = F.log_softmax(preds, dim=1)
        preds_softmax = torch.exp(preds_logsoft)

        preds_softmax = preds_softmax.gather(1, labels)
        preds_logsoft = preds_logsoft.gather(1, labels)

        loss = -torch.mul(torch.pow((1 - preds_softmax), self.gamma), preds_logsoft)

        loss = loss.mean()

        return loss

In [38]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CNNDDI().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = focal_loss()

num_epochs = 50
checkpoint_interval = 10
for epoch in tqdm(range(num_epochs)):
    train_loss = train(model, device, train_loader, optimizer, criterion)
    val_loss, val_accuracy, precision, recall, f1, roc_auc, aupr = validate(model, device, val_loader, criterion)
    print(f'Epoch {epoch+1}, Train Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}, Precision: {precision:.4f}, recall: {recall:.4f}, F1: {f1:.4f}, AUC: {roc_auc:.4f}, AUPR: {aupr:.4f}')

    if (epoch + 1) % checkpoint_interval == 0 or (epoch + 1) == num_epochs:
        save_checkpoint(model, optimizer, epoch + 1, f'checkpoints/cnn_ddi_epoch_{epoch+1}.pt')


  2%|▏         | 1/50 [00:21<17:16, 21.15s/it]

Epoch 1, Train Loss: 0.8764, Validation Loss: 0.3628, Validation Accuracy: 0.7918


  4%|▍         | 2/50 [00:42<16:55, 21.16s/it]

Epoch 2, Train Loss: 0.2777, Validation Loss: 0.3381, Validation Accuracy: 0.8103


  6%|▌         | 3/50 [01:03<16:40, 21.28s/it]

...

Epoch 48, Train Loss: 0.0912, Validation Loss: 2.0154, Validation Accuracy: 0.8968


 98%|█████████▊| 49/50 [17:26<00:21, 21.35s/it]

Epoch 49, Train Loss: 0.2051, Validation Loss: 2.6824, Validation Accuracy: 0.9033
Epoch 50, Train Loss: 0.2484, Validation Loss: 2.6757, Validation Accuracy: 0.8936


100%|██████████| 50/50 [17:49<00:00, 21.38s/it]

Saved checkpoint: checkpoints/cnn_ddi_epoch_50.pt


In [4]:
cnn_ddi = CNNDDI()
state_dict = torch.hub.load_state_dict_from_url("https://huggingface.co/conrevo/CS598-CNN-DDI/resolve/main/cnn_ddi_epoch_50.pt", model_dir="checkpoints")
cnn_ddi.load_state_dict(state_dict["state_dict"])

Downloading: "https://huggingface.co/conrevo/CS598-CNN-DDI/resolve/main/cnn_ddi_epoch_50.pt" to checkpoints/cnn_ddi_epoch_50.pt
100%|██████████| 1.68G/1.68G [00:27<00:00, 66.1MB/s]


<All keys matched successfully>

In [17]:
batch_size = 32
num_epochs = 50
checkpoint_interval = 10
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
for similarity in ["jaccard", "cosine", "gaussian"]:
    print(f"Similarity measure: {similarity}")
    new_feature, new_label, event_num = prepare(df_drug, feature_list, mechanism, action, drugA, drugB, similarity=similarity)
    new_feature = torch.tensor(new_feature, dtype=torch.float32)
    features_train, features_val, labels_train, labels_val = train_test_split(
        new_feature, new_label, test_size=0.2, random_state=42
    )

    # Create dataset objects
    train_dataset = DrugInteractionDataset(features_train, labels_train)
    val_dataset = DrugInteractionDataset(features_val, labels_val)

    # Create data loaders
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    model = CNNDDI().to(device)
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    criterion = focal_loss()

    for epoch in tqdm(range(num_epochs)):
        train_loss = train(model, device, train_loader, optimizer, criterion)
        val_loss, val_accuracy, precision, recall, f1, roc_auc, aupr = validate(model, device, val_loader, criterion)
        print(f'Epoch {epoch+1}, Train Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}, Precision: {precision:.4f}, recall: {recall:.4f}, F1: {f1:.4f}, AUC: {roc_auc:.4f}, AUPR: {aupr:.4f}')

        if (epoch + 1) % checkpoint_interval == 0 or (epoch + 1) == num_epochs:
            save_checkpoint(model, optimizer, epoch + 1, f'checkpoints/cnn_ddi_epoch_{similarity}_{epoch+1}.pt')


Similarity measure: jaccard


/tmp/ipykernel_21746/3982585584.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.features = torch.tensor(features, dtype=torch.float32)
/home/conrevo/miniconda3/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
  2%|▏         | 1/50 [00:21<17:34, 21.51s/it]

Epoch 1, Train Loss: 0.8044, Validation Loss: 0.3486, Validation Accuracy: 0.8075, Precision: 0.6085, recall: 0.5005, F1: 0.5152, AUC: 0.9639, AUPR: 0.9150


  4%|▍         | 2/50 [00:42<17:05, 21.37s/it]

Epoch 2, Train Loss: 0.2814, Validation Loss: 0.2822, Validation Accuracy: 0.8320, Precision: 0.7437, recall: 0.6746, F1: 0.6713, AUC: 0.9751, AUPR: 0.9402


...

Epoch 48, Train Loss: 0.1417, Validation Loss: 2.7146, Validation Accuracy: 0.8731, Precision: 0.7585, recall: 0.7552, F1: 0.7315, AUC: 0.9861, AUPR: 0.9588


 98%|█████████▊| 49/50 [17:58<00:22, 22.25s/it]

Epoch 49, Train Loss: 0.3773, Validation Loss: 2.3333, Validation Accuracy: 0.8912, Precision: 0.7949, recall: 0.7673, F1: 0.7669, AUC: 0.9874, AUPR: 0.9624
Epoch 50, Train Loss: 0.0826, Validation Loss: 2.2991, Validation Accuracy: 0.8921, Precision: 0.7882, recall: 0.8323, F1: 0.7976, AUC: 0.9856, AUPR: 0.9551


100%|██████████| 50/50 [18:21<00:00, 22.04s/it]

Saved checkpoint: checkpoints/cnn_ddi_epoch_gaussian_50.pt


In [8]:
import itertools

feature_list = ["pathway", "target", "enzyme", "category"]

# Generate all combinations for different lengths
all_combinations = []
for r in range(1, len(feature_list) + 1):
    combinations = list(itertools.combinations(feature_list, r))
    all_combinations.extend(combinations)

# Train models for all combinations
batch_size = 32
num_epochs = 50
checkpoint_interval = 10
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
for combination in all_combinations:
    print(f"Feature combination: {combination}")
    new_feature, new_label, event_num = prepare(df_drug, combination, mechanism, action, drugA, drugB, similarity="jaccard")
    new_feature = torch.tensor(new_feature, dtype=torch.float32)
    features_train, features_val, labels_train, labels_val = train_test_split(
        new_feature, new_label, test_size=0.2, random_state=42
    )

    # Create dataset objects
    train_dataset = DrugInteractionDataset(features_train, labels_train)
    val_dataset = DrugInteractionDataset(features_val, labels_val)

    # Create data loaders
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    model = CNNDDI(num_of_features=len(combination)).to(device)
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    criterion = focal_loss()

    for epoch in tqdm(range(num_epochs)):
        train_loss = train(model, device, train_loader, optimizer, criterion)
        val_loss, val_accuracy, precision, recall, f1, roc_auc, aupr = validate(model, device, val_loader, criterion)
        print(f'Epoch {epoch+1}, Train Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}, Precision: {precision:.4f}, recall: {recall:.4f}, F1: {f1:.4f}, AUC: {roc_auc:.4f}, AUPR: {aupr:.4f}')

        if (epoch + 1) % checkpoint_interval == 0 or (epoch + 1) == num_epochs:
            combination_str = "_".join(combination)
            save_checkpoint(model, optimizer, epoch + 1, f'checkpoints/cnn_ddi_epoch_{epoch+1}_{combination_str}.pt')


Feature combination: ('pathway',)


/tmp/ipykernel_83239/3982585584.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.features = torch.tensor(features, dtype=torch.float32)
  2%|▏         | 1/50 [00:05<04:47,  5.87s/it]

Epoch 1, Train Loss: 0.7775, Validation Loss: 0.4056, Validation Accuracy: 0.7588, Precision: 0.5696, recall: 0.5454, F1: 0.5346, AUC: 0.9528, AUPR: 0.8836


  4%|▍         | 2/50 [00:11<04:42,  5.88s/it]

Epoch 2, Train Loss: 0.3502, Validation Loss: 0.3975, Validation Accuracy: 0.7589, Precision: 0.6675, recall: 0.5555, F1: 0.5758, AUC: 0.9596, AUPR: 0.8995


...

 32%|███▏      | 16/50 [05:42<12:07, 21.39s/it]

Epoch 16, Train Loss: 0.0756, Validation Loss: 0.4969, Validation Accuracy: 0.8653, Precision: 0.6974, recall: 0.7693, F1: 0.7068, AUC: 0.9862, AUPR: 0.9647


 34%|███▍      | 17/50 [06:03<11:45, 21.39s/it]

Epoch 17, Train Loss: 0.2332, Validation Loss: 0.3022, Validation Accuracy: 0.8806, Precision: 0.7884, recall: 0.7154, F1: 0.7324, AUC: 0.9900, AUPR: 0.9732


: 

In [15]:


class CNNDDI_no_residual(nn.Module):
    def __init__(self, num_of_features=4):
        super(CNNDDI_no_residual, self).__init__()
        self.conv1 = nn.Conv2d(2, 64, (3, 1), padding=(1, 0))
        self.conv2 = nn.Conv2d(64, 128, (3, 1), padding=(1, 0))
        self.conv3_1 = nn.Conv2d(128, 128, (3, 1), padding=(1, 0))
        self.conv3_2 = nn.Conv2d(128, 128, (3, 1), padding=(1, 0))
        self.conv4 = nn.Conv2d(128, 256, (3, 1), padding=(1, 0))
        self.fc1 = nn.Linear(256 * 572 * num_of_features, 256)  # Adjust feature_size based on your input dimensions
        self.fc2 = nn.Linear(256, 65)  # Assuming 65 DDI types

    def forward(self, x):
        x = F.leaky_relu(self.conv1(x), negative_slope=0.2)
        x = F.leaky_relu(self.conv2(x), negative_slope=0.2)
        # identity = x
        x = F.leaky_relu(self.conv3_1(x), negative_slope=0.2)
        x = self.conv3_2(x)
        # x += identity
        x = F.leaky_relu(x, negative_slope=0.2)
        x = F.leaky_relu(self.conv4(x), negative_slope=0.2)
        x = torch.flatten(x, 1)
        x = F.leaky_relu(self.fc1(x), negative_slope=0.2)
        x = self.fc2(x)
        return x


In [12]:
import copy
import logging
import sys


class ColoredFormatter(logging.Formatter):
    COLORS = {
        "DEBUG": "\033[0;36m",  # CYAN
        "INFO": "\033[0;32m",  # GREEN
        "WARNING": "\033[0;33m",  # YELLOW
        "ERROR": "\033[0;31m",  # RED
        "CRITICAL": "\033[0;37;41m",  # WHITE ON RED
        "RESET": "\033[0m",  # RESET COLOR
    }

    def format(self, record):
        colored_record = copy.copy(record)
        levelname = colored_record.levelname
        seq = self.COLORS.get(levelname, self.COLORS["RESET"])
        colored_record.levelname = f"{seq}{levelname}{self.COLORS['RESET']}"
        return super().format(colored_record)


# Create a new logger
logger = logging.getLogger("CNN-DDI")
logger.propagate = False

# Add handler if we don't have one.
if not logger.handlers:
    handler = logging.StreamHandler(sys.stdout)
    handler.setFormatter(
        ColoredFormatter("%(asctime)s - %(name)s - %(levelname)s - %(message)s")
    )
    logger.addHandler(handler)

# Configure logger
loglevel_string = "INFO"
if not loglevel_string:
    loglevel_string = "INFO"
loglevel = getattr(logging, loglevel_string.upper(), None)
logger.setLevel(loglevel)


In [16]:
batch_size = 32
num_epochs = 50
checkpoint_interval = 10
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
for idx, model_class in enumerate([CNNDDI_no_residual, CNNDDI]):
    model_class_str = "CNNDDI" if idx == 1 else "CNNDDI_no_residual"
    logger.info(f"Model Arch: {model_class_str}")
    new_feature, new_label, event_num = prepare(df_drug, feature_list, mechanism, action, drugA, drugB)
    new_feature = torch.tensor(new_feature, dtype=torch.float32)
    features_train, features_val, labels_train, labels_val = train_test_split(
        new_feature, new_label, test_size=0.2, random_state=42
    )

    # Create dataset objects
    train_dataset = DrugInteractionDataset(features_train, labels_train)
    val_dataset = DrugInteractionDataset(features_val, labels_val)

    # Create data loaders
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    model = model_class().to(device)
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    criterion = focal_loss()

    for epoch in tqdm(range(num_epochs)):
        train_loss = train(model, device, train_loader, optimizer, criterion)
        val_loss, val_accuracy, precision, recall, f1, roc_auc, aupr = validate(model, device, val_loader, criterion)
        logger.info(f'Epoch {epoch+1}, Train Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}, Precision: {precision:.4f}, recall: {recall:.4f}, F1: {f1:.4f}, AUC: {roc_auc:.4f}, AUPR: {aupr:.4f}')

        if (epoch + 1) % checkpoint_interval == 0 or (epoch + 1) == num_epochs:
            if idx == 0:
                save_checkpoint(model, optimizer, epoch + 1, f'checkpoints/cnn_ddi_epoch_{model_class_str}_{epoch+1}.pt')


2024-04-23 21:36:52,900 - CNN-DDI - INFO - Model Arch: CNNDDI_no_residual


/tmp/ipykernel_235039/3982585584.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.features = torch.tensor(features, dtype=torch.float32)
  0%|          | 0/50 [00:00<?, ?it/s]

2024-04-23 21:37:17,179 - CNN-DDI - INFO - Epoch 1, Train Loss: 0.8420, Validation Loss: 0.3400, Validation Accuracy: 0.7974, Precision: 0.6306, recall: 0.6034, F1: 0.5802, AUC: 0.9679, AUPR: 0.9231


  2%|▏         | 1/50 [00:21<17:55, 21.95s/it]

2024-04-23 21:37:39,277 - CNN-DDI - INFO - Epoch 2, Train Loss: 0.2852, Validation Loss: 0.2604, Validation Accuracy: 0.8383, Precision: 0.7413, recall: 0.7131, F1: 0.7061, AUC: 0.9780, AUPR: 0.9457


...

 96%|█████████▌| 48/50 [18:03<00:45, 22.64s/it]

2024-04-23 22:14:04,182 - CNN-DDI - INFO - Epoch 49, Train Loss: 0.1857, Validation Loss: 4.2833, Validation Accuracy: 0.8888, Precision: 0.8004, recall: 0.7975, F1: 0.7799, AUC: 0.9860, AUPR: 0.9497


 98%|█████████▊| 49/50 [18:26<00:22, 22.66s/it]

2024-04-23 22:14:27,015 - CNN-DDI - INFO - Epoch 50, Train Loss: 0.2611, Validation Loss: 3.7770, Validation Accuracy: 0.8672, Precision: 0.7706, recall: 0.7461, F1: 0.7430, AUC: 0.9859, AUPR: 0.9477


100%|██████████| 50/50 [18:48<00:00, 22.58s/it]
